In [2]:
#import ee
#ee.Authenticate()

Enter verification code: 4/1ARtbsJqhOmEzzhx6UHfmjVAoOQXfaIe-o7KB5b7phHsT2JcUaD0RIWcRBpY

Successfully saved authorization token.


# Import all the Libraries and Packages

In [3]:
import os
import ee
import eemont
import geemap
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
ee.Initialize()

In [4]:
# Initialize geemap
m = geemap.Map()

# Import Landsat 5, 7, and 8 Image Collections and Algonquin Features

In [5]:
# Extract landsat 5, landsat 7 and landsat 8 image collection and then filter out the unwanted pixels.

band_l8 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
band_l5_l7 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
l8_bands = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(band_l8)
l7_bands = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").select(band_l5_l7)
l5_bands = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").select(band_l5_l7)
# Import Algonquin Park Boundary Shapefile for clipping and fitlering the image collection
Algonquin_Boundary = ee.FeatureCollection("projects/ee-wang25/assets/Algonquin_Boundary").geometry()

#if we also intend to filter the image caputre day of year, use "ee.Filter.dayOfYear(91,250)"
filters = [
    ee.Filter.intersects('.geo', Algonquin_Boundary),
]

l8_bands_filtered = l8_bands.filter(filters)
l7_bands_filtered = l7_bands.filter(filters)
l5_bands_filtered = l5_bands.filter(filters)

In [6]:
# Apply the cloud, shadow and snow mask

def getBit(n):
    # Returns a GEE server-side object representing `int(2^n)`
    return ee.Number(2).pow(n).int()


def addMaskBand(image):   
    qa = image.select("QA_PIXEL")
    
    dilatedCloudBit = getBit(1)
    cirrusBit = getBit(2)
    cloudBit = getBit(3)
    cloudShadowBit = getBit(4)
    snowBit = getBit(5)
    
    # Define the mask by extracting these bits and reclassifying the pixel based on the bit's value
    mask = ee.Image(0)\
        .where(qa.bitwiseAnd(dilatedCloudBit).neq(0), 1)\
        .where(qa.bitwiseAnd(cloudBit).neq(0), 2)\
        .where(qa.bitwiseAnd(cirrusBit).neq(0), 3)\
        .where(qa.bitwiseAnd(cloudShadowBit).neq(0), 4)\
        .where(qa.bitwiseAnd(snowBit).neq(0), 5)\
        .updateMask(image.select('QA_PIXEL').mask())\
        .rename("cloud_shadow_snow_mask")
    
    # return original image with this mask added as an extra band
    return image.addBands(mask)

l8_bands_filtered_masked = ee.ImageCollection(l8_bands_filtered).map(addMaskBand)
l7_bands_filtered_masked = ee.ImageCollection(l7_bands_filtered).map(addMaskBand)
l5_bands_filtered_masked = ee.ImageCollection(l5_bands_filtered).map(addMaskBand)

def maskImage(image):
    cloud_shadow_snow = image.select("cloud_shadow_snow_mask")
    return image.updateMask(cloud_shadow_snow.eq(0))

l8_final = l8_bands_filtered_masked.map(maskImage)
l7_final = l7_bands_filtered_masked.map(maskImage)
l5_final = l5_bands_filtered_masked.map(maskImage)

# Calculate the AWEInsh Index for Landsat 5 & Landsat 7 (Feyisa et al, 2014)

In [9]:
# Define an expression according to Feyisa et al, (2014) to calculate the non-shadow wetness index

def calcAWEInsh(image):
    AWEInsh = image.expression(
        "4 * (TM2 - TM5) / 10000 - (0.25 * TM4 + 2.75 * TM7) / 10000", {
            "TM2" : image.select("SR_B2"),
            "TM4" : image.select("SR_B4"),
            "TM5" : image.select("SR_B5"),
            "TM7" : image.select("SR_B7")
    }).rename("AWEInsh")
    return image.addBands(AWEInsh)

In [10]:
# Test run on Landsat 5 image collection
l5_final_AWEInsh = ee.ImageCollection(l5_final).map(calcAWEInsh)
l7_final_AWEInsh = ee.ImageCollection(l7_final).map(calcAWEInsh)

# Test if the image collection does include the AWEInsh band
ee.Image(l5_final_AWEInsh.first()).bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B7',
 'QA_PIXEL',
 'cloud_shadow_snow_mask',
 'AWEInsh']

# Calculate the AWEInsh Index for Landsat 8

In [13]:
# The Thematic Mapper sensor name stayed the same but the band selected according to GEE has been modified to fit Landsat 8

def calcAWEInsh(image):
    AWEInsh = image.expression(
        "4 * (TM2 - TM5) / 10000 - (0.25 * TM4 + 2.75 * TM7) / 10000", {
            "TM2" : image.select("SR_B3"),
            "TM4" : image.select("SR_B5"),
            "TM5" : image.select("SR_B6"),
            "TM7" : image.select("SR_B7")
    }).rename("AWEInsh")
    return image.addBands(AWEInsh)

In [14]:
# Landsat 8
l8_final_AWEInsh = ee.ImageCollection(l8_final).map(calcAWEInsh)

# Test if the image collection does include the AWEInsh band
ee.Image(l8_final_AWEInsh.first()).bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'QA_PIXEL',
 'cloud_shadow_snow_mask',
 'AWEInsh']

# Select the Wetness Bands and merge them together

In [15]:
# Select all the wetness bands

l5_AWEInsh = ee.ImageCollection(l5_final_AWEInsh).select("AWEInsh")
l7_AWEInsh = ee.ImageCollection(l7_final_AWEInsh).select("AWEInsh")
l8_AWEInsh = ee.ImageCollection(l8_final_AWEInsh).select("AWEInsh")

# Merge the Wetness Index for Landsat 5, 7, 8
l5_7_AWEInsh = l5_AWEInsh.merge(l7_AWEInsh)
l5_7_8_AWEInsh = l5_7_AWEInsh.merge(l8_AWEInsh)

print(l5_7_8_AWEInsh.size().getInfo())

3782


In [10]:
# Display the wetness index and change the min and max parameter

AWEInshParams = {
  'min': -2500,
  'max': 2500,
  'palette': ['#762a83','#9970ab','#c2a5cf','#e7d4e8','#f7f7f7','#d9f0d3','#a6dba0','#5aae61','#1b7837']
}

m = geemap.Map()
m.addLayer(l5_7_8_AWEInsh, AWEInshParams, "Landsat AWEInsh")
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# Using Geopandas to Loop through Shapefile and Export AWEInsh Value

In [16]:
import geopandas as gpd

# Use ArcGIS Pro to modify this file or to select more pixels
# ArcGIS Pro: Upperstream Location
# https://www.lioapplications.lrc.gov.on.ca/OWIT/index.html?viewer=OWIT.OWIT&locale=en-CA
# Note: For Test_Upperstream_Location, the attribute "id" is now "Id". Make sure to change that in the test run.
upstream = gpd.read_file("D:/Yannans Stuff/Data/Visible Dam/Visible Dam.shp")
upstream

,Id,geometry
0,0,POINT (-78.68041 45.69172)
1,1,POINT (-78.67734 45.69231)
2,2,POINT (-78.67780 45.68798)


In [17]:
for i in upstream.index:
    print(upstream.loc[i,'Id'])

0
1
2


In [18]:
# outdir = "D:/Yannans Stuff/Data/NDWI_Export" # specify an output directory
# os.makedirs(outdir, exist_ok = True) ## create it if it doesn't already exist

In [19]:
for i in upstream.index:
    # Here we will convert each individual point into a ee.geometry
    gpd_geom = upstream.loc[i,'geometry']
    x = gpd_geom.xy[0][0]
    y = gpd_geom.xy[1][0]
    ee_geom = ee.Geometry.Point([x,y])
    
    # Followed by the eemont code
    ts = l5_7_8_AWEInsh.getTimeSeriesByRegion(geometry = ee_geom,
                               reducer = ee.Reducer.mean(),
                               scale = 30)
    tsPandas = geemap.ee_to_pandas(ts)
    tsPandas[tsPandas == -9999] = np.nan
    tsPandas = tsPandas.dropna()
    
    #Create an output label using the output directory and the 'id' column
    ID = upstream.loc[i,'Id']
    tsPandas.to_csv(r"D:\Yannans Stuff\Data\Visible Dam\Landsat5_7_8_AWEInsh_TS_{0}.csv".format(ID))